In [34]:
import gzip
from collections import defaultdict
import numpy
from numpy.linalg import inv
import random

def readGz(f):
  for l in gzip.open(f):
    yield eval(l)

In [33]:
### Rating
raw_data = []
for l in readGz("train.json.gz"):
    user, item = l['reviewerID'], l['itemID']
    categoryID = l['categoryID']
    rating = l['rating']
    reviewText = l['reviewText']
    raw_data.append([user, item, categoryID, rating, l['helpful'], reviewText])

train_data = raw_data[:200000]
valid_data = raw_data[100000:]

userRatings = defaultdict(list)
itemRatings = defaultdict(list)

In [14]:
# calculate average
for l in train_data:
    user = l[0]
    item = l[1]
    rating = l[3]        
    userRatings[user].append(rating)
    itemRatings[item].append(rating)

userRatings_ave = defaultdict(dict)
itemRatings_ave = defaultdict(dict)
for user in userRatings:
    userRatings_ave[user] = numpy.mean(userRatings[user])
for item in itemRatings:
    itemRatings_ave[item] = numpy.mean(itemRatings[item])

# AS1

In [35]:
import gzip
from collections import defaultdict
import numpy
from numpy.linalg import inv
import random

def readGz(f):
  for l in gzip.open(f):
    yield eval(l)

In [36]:
### Rating
allRatings = []
userRatings = defaultdict(list)
itemRatings = defaultdict(list)
for l in readGz("train.json.gz"):
    user, item = l['reviewerID'], l['itemID']
    allRatings.append(l['rating'])
    userRatings[user].append(l['rating'])
    itemRatings[item].append(l['rating'])
    

In [37]:
userRatings_ave = defaultdict(dict)
itemRatings_ave = defaultdict(dict)
for user in userRatings:
    userRatings_ave[user] = numpy.mean(userRatings[user])
for item in itemRatings:
    itemRatings_ave[item] = numpy.mean(itemRatings[item])


In [150]:
'''
allRatings = []
userRatings = defaultdict(list)
itemRatings = defaultdict(list)
for l in readGz("train.json.gz"):
    user, item = l['reviewerID'], l['itemID']
    
    if abs(userRatings_ave[l['reviewerID']] - l['rating']) > 2 and abs(itemRatings_ave[l['itemID']] - l['rating']) > 2:
        continue
    allRatings.append(l['rating'])
    userRatings[user].append(l['rating'])
    itemRatings[item].append(l['rating'])
userRatings_ave = defaultdict(dict)
itemRatings_ave = defaultdict(dict)
for user in userRatings:
    userRatings_ave[user] = numpy.mean(userRatings[user])
for item in itemRatings:
    itemRatings_ave[item] = numpy.mean(itemRatings[item])
'''

In [39]:
train_Ratings = allRatings[:200000]
valid_Ratings = allRatings[100000:]

In [40]:
len(train_Ratings)

200000

In [41]:
# problem 5
globalAverage = sum(train_Ratings) * 1.0 / len(train_Ratings)
print "alpha = " + str(globalAverage)
MSE_rating_valid = sum([(i - globalAverage) ** 2 for i in valid_Ratings]) * 1.0 / len(valid_Ratings)
print "MSE on valid set is " + str(MSE_rating_valid)

alpha = 4.23359
MSE on valid set is 1.2264635521


In [42]:
allRatings = []
userRatings = defaultdict(list)
for l in readGz("train.json.gz"):
    '''
    if abs(userRatings_ave[l['reviewerID']] - l['rating']) > 2 and abs(itemRatings_ave[l['itemID']] - l['rating']) > 2:
        continue
    '''
    allRatings.append([l['reviewerID'], l['itemID'], l['rating']])
train_Ratings = allRatings[:200000]
valid_Ratings = allRatings[100000:]

In [43]:
R_u = defaultdict(dict)
R_i = defaultdict(dict)
users = {}
items = {}
gamma_u = {}
gamma_i = {}
K = 5

for l in train_Ratings:
    user, item, rating = l[0], l[1], l[2]
#    if (user in R_u) and (item in R_u[user]):
#        continue
    
    R_u[user][item] = rating
    R_i[item][user] = rating
    if user not in users:
        users[user] = 0.0
        gamma_u[user] = numpy.random.random_sample((K, 1)) * 0.5
    if item not in items:
        items[item] = 0.0
        gamma_i[item] = numpy.random.random_sample((K, 1)) * 0.5

In [44]:
def calc_alpha():
    sigma = 0
    count = 0
    for user in users:
        g_u = gamma_u[user]
        u = users[user]
        for item in R_u[user]:
            sigma += R_u[user][item] - (u + items[item] + numpy.dot(g_u.T, gamma_i[item]))
            count += 1
    return sigma * 1.0 / count

In [45]:
def calc_users():
    users_tmp = {}
    for user in users:
        sigma = 0
        count = 0
        g_u = gamma_u[user]
        for item in R_u[user]:
            sigma += R_u[user][item] - (alpha + items[item] + numpy.dot(g_u.T, gamma_i[item]))
            count += 1
        users_tmp[user] = sigma * 1.0 / (lam + count)
    return users_tmp

In [46]:
def calc_items():
    items_tmp = {}
    for item in items:
        sigma = 0
        count = 0
        g_i = gamma_i[item]
        for user in R_i[item]:
            sigma += R_i[item][user] - (alpha + users[user] + numpy.dot(g_i.T, gamma_u[user]))
            count += 1
        items_tmp[item] = sigma * 1.0 / (lam + count)
    return items_tmp

In [47]:
def calc_gamma_u():
    gamma_u_tmp = {}
    for user in users:
        sigma1 = numpy.array([[0.0] * K]) * numpy.array([[0.0] * K]).T
        sigma2 = numpy.array([[0.0] * K]).T       
        u = users[user]
        for item in R_u[user]:
            g_i = gamma_i[item]
            sigma1 += g_i * g_i.T
            sigma2 += g_i * (R_u[user][item] - (alpha + items[item] + u))
        gamma_u_tmp[user] = numpy.dot(inv(sigma1 + 2*lam*numpy.eye(K)), sigma2)
    return gamma_u_tmp

In [48]:
def calc_gamma_i():
    gamma_i_tmp = {}
    for item in items:
        sigma1 = numpy.array([[0.0] * K]) * numpy.array([[0.0] * K]).T
        sigma2 = numpy.array([[0.0] * K]).T
        i = items[item]
        for user in R_i[item]:
            g_u = gamma_u[user]
            sigma1 += g_u * g_u.T
            sigma2 += g_u * (R_u[user][item] - (alpha + users[user] + i))
        gamma_i_tmp[item] = numpy.dot(inv(sigma1 + 2*lam*numpy.eye(K)), sigma2)
    return gamma_i_tmp

In [49]:
def calc_target(alpha_cur, users_cur, items_cur, gamma_u_cur, gamma_i_cur, data, lam):
    sigma = 0    
    for user in users_cur:
        sigma += users_cur[user] ** 2 + numpy.dot(gamma_u_cur[user].T, gamma_u_cur[user])
    for item in items_cur:
        sigma += items_cur[item] ** 2 + numpy.dot(gamma_i_cur[item].T, gamma_i_cur[item])
    sigma *= lam
    for l in data:
        sigma += (alpha_cur + users_cur[l[0]] + items_cur[l[1]] + numpy.dot(gamma_u_cur[l[0]].T, gamma_i_cur[l[1]])- l[2]) ** 2
    return sigma

In [50]:
gamma_u['U999710097']

array([[ 0.04162581],
       [ 0.1575244 ],
       [ 0.40213984],
       [ 0.13964372],
       [ 0.48376819]])

In [51]:
def calc_MSE(alpha_cur, users_cur, items_cur, gamma_u_cur, gamma_i_cur, data, default_rating):
    sigma = 0
    for l in data:
        if l[0] not in users_cur:
            b_u = 0
        else:
            b_u = users_cur[l[0]]
        if l[1] not in items_cur:
            b_i = 0
        else:
            b_i = items_cur[l[1]]
        if l[0] not in gamma_u_cur:
            r_u = numpy.array([[0.0] * K]).T 
        else:
            r_u = gamma_u_cur[l[0]]
        if l[1] not in gamma_i_cur:
            r_i = numpy.array([[0.0] * K]).T
        else:
            r_i = gamma_i_cur[l[1]]
        sigma += (alpha_cur + b_u + b_i + numpy.dot(r_u.T, r_i) - l[2]) ** 2
    return sigma * 1.0 / len(data)

In [52]:
globalAverage = sum(l[2] for l in train_Ratings) * 1.0 / len(train_Ratings)

In [53]:
globalAverage

4.23359

In [54]:
alpha = globalAverage

In [55]:
lam = 5.5
target_new = calc_target(alpha, users, items, gamma_u, gamma_i, train_Ratings, lam)
target = target_new + 100  # Tricky: Compel target to be larger than target_new
globalAverage = sum(l[2] for l in train_Ratings) * 1.0 / len(train_Ratings)
trial = 1
while abs(target - target_new) > 1:
    while abs(target - target_new) > 1:
        target = target_new
        alpha = calc_alpha()
        users = calc_users()
        items = calc_items()
        gamma_u = calc_gamma_u()
        target_new = calc_target(alpha, users, items, gamma_u, gamma_i, train_Ratings, lam)
        trial += 1
        print trial
    gamma_i = calc_gamma_i()
    target_new = calc_target(alpha, users, items, gamma_u, gamma_i, train_Ratings, lam)
    while abs(target - target_new) > 1:
        target = target_new
        alpha = calc_alpha()
        users = calc_users()
        items = calc_items()
        gamma_i = calc_gamma_i()
        target_new = calc_target(alpha, users, items, gamma_u, gamma_i, train_Ratings, lam)
        trial += 1
        print trial
    gamma_u = calc_gamma_u()  
    target_new = calc_target(alpha, users, items, gamma_u, gamma_i, train_Ratings, lam)


    

2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67


In [56]:
min_users = users
min_items = items
min_alpha = alpha
min_gamma_u = gamma_u
min_gamma_i = gamma_i

In [57]:
MSE_valid_6 = calc_MSE(alpha, users, items, gamma_u, gamma_i, valid_Ratings, globalAverage)
print "MSE in problem6 = " + str(MSE_valid_6)

MSE in problem6 = [[ 0.79739667]]


In [95]:
min_users_ave = sum(min_users[d] for d in min_users) / len(min_users)
min_items_ave = sum(min_items[d] for d in min_items) / len(min_items)
min_gamma_u_ave = sum(min_gamma_u[d] for d in min_gamma_u) / len(min_gamma_u)
min_gamma_i_ave = sum(min_gamma_i[d] for d in min_gamma_i) / len(min_gamma_i)
predictions = open("predictions_Rating.csv", 'w')
for l in open("pairs_Rating.txt"):
    if l.startswith("userID"):
        #header
        predictions.write(l)
        continue
    u, i = l.strip().split('-')
    
    if (u not in min_users) and (i not in min_items):
        predictions.write(u + '-' + i + ',' + str(globalAverage) + '\n')
        continue
    if u in min_users and i in R_u[u]:
        predictions.write(u + '-' + i + ',' + str(R_u[u][i]) + '\n')
        continue
    if u in min_users and i in min_items:
        beta_u = min_users[u]
        ga_u = min_gamma_u[u]
        beta_i = min_items[i]
        ga_i = min_gamma_i[i]
        if (beta_u + beta_i + min_alpha + numpy.dot(ga_u.T, ga_i))[0][0] < 1.9:
            predictions.write(u + '-' + i + ',' + str(1) + '\n')
        elif (beta_u + beta_i + min_alpha + numpy.dot(ga_u.T, ga_i))[0][0] >= 4.7:
            predictions.write(u + '-' + i + ',' + str(4.7) + '\n')
        else:
            predictions.write(u + '-' + i + ',' + str((beta_u + beta_i + min_alpha + numpy.dot(ga_u.T, ga_i))[0][0]) + '\n')
        continue
    if u in min_users:
        predictions.write(u + '-' + i + ',' + str(userRatings_ave[u] ) + '\n')
        continue
    if i in min_items:
        predictions.write(u + '-' + i + ',' + str(itemRatings_ave[i] ) + '\n')
'''
    if u in min_users:
        beta_u = min_users[u]
        ga_u = min_gamma_u[u]
    else:
        beta_u = min_users_ave
        ga_u = min_gamma_u_ave
    if i in min_items:
        beta_i = min_items[i]
        ga_i = min_gamma_i[i]
    else:
        beta_i = min_items_ave
        ga_i = min_gamma_i_ave

    predictions.write(u + '-' + i + ',' + str((beta_u + beta_i + min_alpha + numpy.dot(ga_u.T, ga_i))[0][0]) + '\n')
'''
predictions.close()

In [27]:
min_alpha

array([[ 4.23011935]])

In [234]:
min_MSE = MSE_valid_6
min_lam = 5
lam_l = 4.5
lam_h = 5.5
myrange = numpy.arange(lam_l, lam_h, 0.1)
for lam in myrange:
    print lam
    target_new = calc_target(alpha, users, items, gamma_u, gamma_i, train_Ratings, lam)
    target = target_new + 100  # Tricky: Compel target to be larger than target_new
    globalAverage = sum(l[2] for l in train_Ratings) * 1.0 / len(train_Ratings)
    trial = 1
    while abs(target - target_new) > 10:
        while abs(target - target_new) > 10:
            target = target_new
            alpha = calc_alpha()
            users = calc_users()
            gamma_u = calc_gamma_u()
            target_new = calc_target(alpha, users, items, gamma_u, gamma_i, train_Ratings, lam)
            trial += 1
            print trial
        gamma_i = calc_gamma_i()
        target_new = calc_target(alpha, users, items, gamma_u, gamma_i, train_Ratings, lam)
        while abs(target - target_new) > 10:
            target = target_new
            alpha = calc_alpha()
            users = calc_users()
            gamma_i = calc_gamma_i()
            target_new = calc_target(alpha, users, items, gamma_u, gamma_i, train_Ratings, lam)
            trial += 1
            print trial
        gamma_u = calc_gamma_u()  
        target_new = calc_target(alpha, users, items, gamma_u, gamma_i, train_Ratings, lam)
    MSE_valid_new = calc_MSE(alpha, users, items, gamma_u, gamma_i, valid_Ratings, globalAverage)
    print str(MSE_valid_new)
    if MSE_valid_new < min_MSE:
        min_lam = lam
        min_MSE = MSE_valid_new
        min_users = users
        min_items = items
        min_alpha = alpha
        min_gamma_u = gamma_u
        min_gamma_i = gamma_i
        print "best: " + str(lam)

4.5
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
[[ 0.88252577]]
best: 4.5
4.6
2
3
4
5
6
7
8
[[ 0.88651635]]
4.7
2
3
4
5


KeyboardInterrupt: 

# Last HW

In [ ]:
# Problem 7
user_max = max(users, key=users.get)
user_min = min(users, key=users.get)
item_max = max(items, key=items.get)
item_min = min(items, key=items.get)
print "user_max = " + str(user_max)
print "user_min = " + str(user_min)
print "item_max = " + str(item_max)
print "item_min = " + str(item_min)

# Problem 8
min_MSE = MSE_valid_6
min_lam = 1
lam_l = 1
lam_h = 10
myrange = numpy.arange(lam_l, lam_h, 7)
for lam in myrange:
    alpha = 4.23198
    target_new = calc_target(alpha, users, items, gamma_u, gamma_i, train_Ratings, lam)
    target = target_new + 100
    globalAverage = sum(l[2] for l in train_Ratings) * 1.0 / len(train_Ratings)
    trial = 1
    while abs(target - target_new) > 1e-9:
        target = target_new
        if trial % 3 == 0:
            alpha = calc_alpha()
        elif trial % 3 == 1:
            users = calc_users()
        else:
            items = calc_items()
        target_new = calc_target(alpha, users, items, gamma_u, gamma_i,train_Ratings, lam)
        trial += 1
        #print trial
    print lam
    MSE_valid_8 = calc_MSE(alpha, users, items, valid_Ratings, globalAverage)
    if MSE_valid_8 < min_MSE:
        min_MSE = MSE_valid_8
        min_lam = lam
        min_users = users
        min_items = items
        min_alpha = alpha
        print str(lam) + "MSE = " + str(MSE_valid_8)

In [213]:
sum(min_gamma_u[d] for d in min_gamma_u) / len(min_gamma_u)

array([[  1.54938326e-05],
       [ -3.63948939e-05],
       [ -2.54616277e-05]])

In [44]:
min_users_ave = sum(min_users[d] for d in min_users) / len(min_users)
min_items_ave = sum(min_items[d] for d in min_items) / len(min_items)
min_gamma_u_ave = sum(min_gamma_u[d] for d in min_gamma_u) / len(min_gamma_u)
min_gamma_i_ave = sum(min_gamma_i[d] for d in min_gamma_i) / len(min_gamma_i)
predictions = open("predictions_Rating.csv", 'w')
for l in open("pairs_Rating.txt"):
    if l.startswith("userID"):
        #header
        predictions.write(l)
        continue
    u, i = l.strip().split('-')
    if u in min_users:
        beta_u = min_users[u]
        ga_u = min_gamma_u[u]
    else:
        beta_u = min_users_ave
        ga_u = min_gamma_u_ave
    if i in min_items:
        beta_i = min_items[i]
        ga_i = min_gamma_i[i]
    else:
        beta_i = min_items_ave
        ga_i = min_gamma_i_ave

    predictions.write(u + '-' + i + ',' + str((beta_u + beta_i + min_alpha + numpy.dot(ga_u.T, ga_i))[0][0]) + '\n')

predictions.close()

In [204]:
numpy.dot(ga_u.T, ga_i)[0][0]

-1.4409322442346891e-12

In [205]:
min_alpha

array([[ 4.23069069]])

In [207]:
beta_u

array([[ 0.21980266]])